<a href="https://colab.research.google.com/github/Nazhan99/Tensorflow_notes/blob/main/10_time_series_forecasting_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time series forecasting fundamentals with tensorflow + milestone project 3: BitPredict

let's write some tensorflow code to predict the price of Bitcoin based on the historical price of Bitcoin

In [ ]:
## Get data
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv